In [ ]:
import glob
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import torch
from torch import nn
import random

# Personnal Import 
from utilities import DataSet, get_batch,get_mode_date2path, str2neg, str_xa2int
from dl_models.graph_conv import graphconv

# Data
- Validation individuelles, aggrégée 3min.
- Metro 15 min (entrée/sortie)

Idée : Identifier des coupure de métro longue :
    - entrée et sortie inhabituelles, montre que le métro est suspendu et laisse les portes ouvertes
    - Sortie plus faible (coupure > 15min)
Evaluer les qualités de prédiction sur ces moments là.

## Data Description
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. 
    - Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.

#### Questionnement 
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

## Load Validation Individuelles

In [ ]:
folder_path = 'data/'
valid_ind_path = folder_path + 'Sub_Tram_11_2019_03_2020'
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(folder_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(folder_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])


#### Load 3 df : df_sub, df_tram, df_bus

In [ ]:
for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

## Load Subway 15 min

In [ ]:
df_subway_path_2019_2020 = "../../Data/keolis_data_2019-2020/Métro 15 minutes 2019 2020.txt"
df_metro_funi_2019_2020 = pd.read_csv(df_subway_path_2019_2020, delimiter ="\t",low_memory=False).rename(columns = {' Date jour (CAS tr)':'date','Heure (CAS tr)':'hour','Nb entrées total (CAS tr)':'in','Nb sorties total (CAS tr)':'out'})

# Tackle float values represented by str: 
df_metro_funi_2019_2020['out'] = df_metro_funi_2019_2020['out'].transform(lambda x : str_xa2int(x))
df_metro_funi_2019_2020['in'] = df_metro_funi_2019_2020['in'].transform(lambda x : str_xa2int(x))

# Tackle String Values ('A', 'B'):
df_metro_funi_2019_2020['in'] = df_metro_funi_2019_2020['in'].transform(lambda x : str2neg(x))  # Transform each string values like 'A', 'B', '1A'  and so on to '-1'. Keep NaN values as NaN values.
df_metro_funi_2019_2020['out'] = df_metro_funi_2019_2020['out'].transform(lambda x : str2neg(x))  

# Add '20' like '01_01_2020' instead of '01_01_20'
df_metro_funi_2019_2020['date'] = df_metro_funi_2019_2020['date'].transform(lambda d : d+'20' if len(d)<10 else d)
T_2020 = pd.to_datetime(df_metro_funi_2019_2020['date'] + ' ' + df_metro_funi_2019_2020['hour'], format='%d/%m/%Y %H:%M', errors='coerce')
df_metro_funi_2019_2020['datetime'] = pd.to_datetime(df_metro_funi_2019_2020['date'] + ' ' + df_metro_funi_2019_2020['hour'], format='%d/%m/%Y %H:%M:%S', errors='coerce').fillna(T_2020)

# Keep usefull columns
df_metro_funi_2019_2020 = df_metro_funi_2019_2020[['datetime','Station','Code ligne','in','out']]

# Restrain to usefull 'dates' : 
start,end = datetime(int(dates[0].split('-')[1]),int(dates[0].split('-')[0]),1),datetime(int(dates[-1].split('-')[1]),int(dates[-1].split('-')[0])+1,1)
df_metro_funi_2019_2020 = df_metro_funi_2019_2020[(df_metro_funi_2019_2020.datetime >= start) & (df_metro_funi_2019_2020.datetime < end)]

df_metro_funi_2019_2020.head()

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.axes_grid1 as axes_grid1
import pandas as pd
import numpy as np

def plot_coverage_matshow(data, x_labels = None, y_labels = None, log = False, cmap ="afmhot", save = None, cbar_label =  "Number of Data"):
    # Def function to plot a df with matshow
    # Use : plot the coverage through week and days 

    if log : 
        data = np.log(data + 1)
    
    data[data == 0] = np.nan

    fig = plt.figure(figsize=(40, 12))
    cax = plt.matshow(data.values, cmap=cmap)  #

    cmap_perso = plt.get_cmap(cmap)
    cmap_perso.set_bad('gray', 1.0)  # Configurez la couleur grise pour les valeurs nulles

    # Configurez la colormap pour gérer les valeurs NaN comme le gris
    cax.set_cmap(cmap_perso)
    cax.set_clim(vmin=0.001, vmax=data.max().max())  # Ajustez les limites pour exclure les NaN


    #x labels
    if x_labels is None:
        x_labels = data.columns.values
    plt.gca().set_xticks(range(len(x_labels)))
    plt.gca().set_xticklabels(x_labels, rotation=85, fontsize=8)
    plt.gca().xaxis.set_ticks_position('bottom')

    #y labels
    if y_labels is None: 
        y_labels = data.index.values
    plt.gca().set_yticks(range(len(y_labels)))
    plt.gca().set_yticklabels(y_labels, fontsize=8)

    # Add a colorbar to the right of the figure
    cbar = plt.colorbar(cax, aspect=10)
    cbar.set_label(cbar_label)  # You can customize the label as needed

    if save is not None: 
            plt.savefig(save, format="pdf")
            
def coverage_day_month(df_metro,freq= '24h',index = 'month_year',columns = 'day_date',save = 'subway_id',folder_save = 'save/'):

    df_agg = df_metro.groupby([pd.Grouper(key = 'datetime',freq = freq)]).agg('sum').reset_index()[['datetime','in','out']]
    df_agg['date']= df_agg.datetime.dt.date
    df_agg['day_date'] = df_agg.datetime.dt.day
    df_agg['month_year']= df_agg.datetime.dt.month.transform(lambda x : str(x)) + ' ' + df_agg.datetime.dt.year.transform(lambda x : str(x))
    df_agg['month_year']= pd.to_datetime(df_agg['month_year'],format = '%m %Y')
    #df_agg['hour']= df_agg.datetime.dt.hour.transform(lambda x : str(x)) + ':' + df_agg.datetime.dt.minute.transform(lambda x : str(x))
    df_agg['hour']= df_agg.datetime.dt.hour + df_agg.datetime.dt.minute*0.01
    df_agg['tot'] = df_agg['in'] + df_agg['out']
    # Pivot

    df_agg_in = df_agg.pivot(index = index,columns = columns,values = 'in').fillna(0)
    df_agg_out = df_agg.pivot(index = index,columns = columns,values = 'out').fillna(0)
    df_agg_tot = df_agg.pivot(index = index,columns = columns,values = 'tot').fillna(0)
    
    if index == 'month_year':
        df_agg_in.index = df_agg_in.index.strftime('%Y-%m')
        df_agg_out.index = df_agg_out.index.strftime('%Y-%m')
        df_agg_tot.index = df_agg_out.index.strftime('%Y-%m')


    # Plot 
    plot_coverage_matshow(df_agg_in, log  = False, cmap = 'YlOrRd',save = f'{folder_save}in_{save}')   
    plot_coverage_matshow(df_agg_out, log  = False, cmap = 'YlOrRd',save = f'{folder_save}out_{save}')  
    plot_coverage_matshow(df_agg_tot, log  = False, cmap = 'YlOrRd',save = f'{folder_save}tot_{save}')  
    return(df_agg_in,df_agg_out)

#### Visualisation des flux IN et OUT entre 6h et 24H. En virant les outliers type fête des lumières: 

In [ ]:
freq  = '15min'
columns = 'hour'
index = 'date'
quantile = 0.95
è
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    df_tmps = df_tmps[df_tmps.datetime.dt.hour > 5]
    in99,out99 = df_tmps['in'].quantile(quantile),df_tmps['out'].quantile(quantile)
    df_tmps.loc[df_tmps['in']>in99,'in'] = in99
    df_tmps.loc[df_tmps['out']>out99,'out'] = out99
    coverage_day_month(df_tmps, freq = freq,index = index,columns = columns,save = station,folder_save = 'save/profile flux 15min filtred outliers 95/')

#### Visualisation des flux IN et OUT pour chacunes des stations

In [ ]:
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    coverage_day_month(df_tmps, freq = '60min',index = 'date',columns = 'hour',save = station,folder_save = 'save/profile flux')

# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité.
- Identifier des moment interessant : anomalie sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.

## Feature Vector
A définir pour chacune des stations

## Bla Bla : 

In [ ]:
#Bijecction entre un entier, et son label
def int2lab(int,n_adj=2,B=3):
    n = 1+(int-1)//B 
    b = int-(n-1)*B
    return(f'n{n}_b{b}')

def lab2int(lab,n_adj=2,B=3):
    nb = lab.split('_')
    n,b = int(nb[0][1:]),int(nb[1][1:])
    return((n-1)*B+b)

for k in range(1,15):
    print(int2lab(k),lab2int(int2lab(k)))

## Représentation des numéro des cellules du Tensor
Permet de faire des affichages graphique, et de s'assurer que les ".reshape" font ce que l'on souhaite

In [ ]:
def int2lab(integer, n_adj=2, B=3, C=4, N=5, L=6):
    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    l = 1 + ((integer - 1) % L)
    remaining = (integer - 1) // L
    n = 1 + (remaining % N)
    remaining //= N
    c = 1 + (remaining % C)
    remaining //= C
    b = 1 + (remaining % B)
    remaining //= B
    n_adj = 1 + remaining % n_adj

    return f'adj{n_adj}_b{b}_c{c}_n{n}_l{l}'

def lab2int(label, n_adj=2, B=3, C=4, N=5, L=6):
    split_label = label.split('_')
    n_adj = int(split_label[0][3:])
    b = int(split_label[1][1:])
    c = int(split_label[2][1:])
    n = int(split_label[3][1:])
    l = int(split_label[4][1:])

    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    integer = ((l - 1) + (n - 1) * L + (c - 1) * N * L + (b - 1) * C * N * L + (n_adj - 1) * B * C * N * L) + 1
    return integer


for k in range(1,400):
    print(int2lab(k),lab2int(int2lab(k)))

In [ ]:
label = int2lab(k)
label.split('_')

In [ ]:
permuted_H = output_H.permute(0, 1, 3, 4, 2)
print(permuted_H.shape)

new_c_in = permuted_H.shape[-1]
first_attn = permuted_H.reshape(K,-1,new_c_in)
print(first_attn.shape)  # [K, B*L*N, C_in']

attn_weight = nn.Linear(new_c_in,1)
softmax = nn.Softmax(-1)  #SoftMax on the last dimension 

#Coefficient d'attention 
attn = attn_weight(first_attn)    # "Embedding" du spatial channel 
attn = attn.permute(1,2,0)  # Permute pour avoir la nombre d'adjacency matrix en dernière dimension
attn = softmax(attn)  # Coefficient d'attention pour chaque Matrices d'adjacence et Embedding spatial assicié  (Ici 1 seule matrice d'adjacence)